# Consolidation Exercise 1 - Autonomous Robot Navigation

So far, we've covered:

+ Variables, data types and operators
+ If-then-else
+ Loops (for, while)
+ Functions

This week, we'll be doing our very first *consolidation* exercise. In prior weeks, we've been looking at specific elements of Python syntax. This is necessary, but has the drawback that when you're solving the exercises, you're given a big clue about what you should do- e.g. in week 3, when we looked at loops, there's a fairly good chance that you'll need to use loops to solve the exercises. 

The purpose of consolidation exercises is to give you practice writing code in a scenario that is closer to what you'll face after this course, where it won't always be immediately obvious how a piece of code should be written. This practice will be invaluable after the course when your programming *in the wild*, but at first you may find it hard to get started- given such a big problem, what should you do first? My advice is to start by simplifying the problem and building your solution up bit-by-bit. As the old adage goes, the only way to eat an elephant is one bite at a time!

Consolidation exercises are also a chance for you to write slightly longer pieces of code than we'll generally have time to do in the lab. This means you'll need to think much more about how to structure and organise your code.

You have hopefully also noticed that there's no pre-learning materials or comprehension checks for this week; the flip-side of that is that we expect you to spend slightly more time (a few hours) working on these problems that you would a regular lab sheet. Consolidation exercises are also much closer to what we would expect you to complete for your individual programming project (worth 80% of your mark in this course) and so represent the best chance for you to "practice" the coursework before you complete it. As such, we'll be providing both formative and peer feedback for consolidation exercises submitted on Blackboard.

## Overview

We've been building a piece of code for simulating the movement of a two-wheeled robot.

<img src="https://github.com/engmaths/SEMT10002_2025/blob/main/media/week_1/romi.jpg?raw=true" width="25%" />
<img src="https://github.com/engmaths/SEMT10002_2025/blob/main/media/week_1/robot_kinematics_2.png?raw=true" width="30%" style="background-color: white;" />

So far you've written some code for storing information, and for calculating a new position when given left and right wheel speeds and a timestep. You've also seen how to organise this code into functions, and written a loop to execute a specific movement pattern. Today, we'll extend the functionality of this simulation by adding:

1. Collision detection / avoidance.
2. Basic goal-finding capabilities.
3. Insect-inspired navigation.


## Preliminaries

We've edited the plotting library to now include functionality for plotting the boundaries of a map, an obstacles (if it exists), and a goal position (if it exists). We've also edited the show_plot function to optionally take as input a parameter called "pause". If pause is not provided, show_plot will work as before (by showing you a static image of the robot's position). Each time you call show_plot, the image will remain there until you close it. If, instead, you call show_plot with a value provided, then the image will only show for the number of seconds pause is equal to- in our example, the image will show for 0.1 seconds. This lets us call show_plot repeatedly within a loop to generate an animation of the robot moving around.

If you look in the file *robot_plotter.py*, you'll see we've done this by changing the function *header* (i.e the first line of the function definition). 

```python
def show_plot(map_coords, goal=None, obstacle=None, pause=0):
```

As a reminder, this is defining a function with 4 inputs- ```map_coords, goal, obstacle, pause```. You'll also notice that while the first input (```map_coords```) is defined as normal, the other 3 have values after them (i.e ```goal=None```). Here, we're using Python syntax that allows us to define a *default* value for an input to a function. This has a secondary effect of making that input *optional*. This means if we call the function with just one input:

```python 
show_plot(map_coordinates)
```

then ```goal``` and ```obstacle``` will equal ```None```, while ```pause``` will equal 0. This will generate an image that looks like:

<img src="https://raw.githubusercontent.com/engmaths/SEMT10002_2025/refs/heads/main/media/week_5/map_coords.png" width="40%"/>

On the other hand, if we call the function with two inputs:

```python
show_plot(map_coordinates, (100, 100))
```

then ```goal``` will be overwritten and will now be equal to ```(100, 100)```. This will create an image like:

<img src="https://raw.githubusercontent.com/engmaths/SEMT10002_2025/refs/heads/main/media/week_5/map_and_goal.png" width="40%" />

What if we want to use the default value for ```goal```, but not ```obstacle```? Python will let us do this if we *name* the variable we are providing as input. This is done as below:

```python
show_plot(map_coordinates, obstacle=((-2400, -250), (4000, 500)))
```
Here, ```obstacle``` will be ```((-2400, -250), (4000, 500))```, but ```goal``` and ```pause``` will use the default values. This will create an image like:

<img src="https://raw.githubusercontent.com/engmaths/SEMT10002_2025/refs/heads/main/media/week_5/map_and_obstacle.png" width="40%"/>

Finally, if we overwrite both ```goal``` and ```obstacle```, then we'll get an image like:

<img src="https://raw.githubusercontent.com/engmaths/SEMT10002_2025/refs/heads/main/media/week_5/map_goal_and_obstacle.png" width="40%"/>

The show_plot function expects the map coordinates to be defined by four values:

```python
map_coords = ((map_x_min, map_x_max), (map_y_min, map_y_max))
```

and obstacles to be defined as four values:

```python
obstacle = ((x_position, y_position), (width, height))
```

In both cases, we are using Python objects known as *tuples*. We'll cover these fully in a couple of weeks- for now, all you need to know is that a bit like a string, we can access elements of a tuple by providing an index within square brackets- i.e obstacle[0] will give us the first element of the obstacle (i.e (x_position, y_position)), while obstacle[1] will give us the second element. We're nesting tuples in both cases, so to get e.g. the x_position of the obstacle we would use obstacle[0][0]. 



Finally, we can put all of this together with some of the code we've already written that to create a simulation that will visualise our robot as it drives around randomly. We're doing this by using the function ```random``` which we've imported from the library *random*. This function will generate a random value between 0 and 1 and lets us set the wheel velocities randomly.

Running the code below from the notebook will only work if you've got the "robot_plotter.py" file in the same directory as this notebook file. To make it easier, we've put some this code into a file "robot_sim.py" that you should be able to download and run from Blackboard, although note it'll still need to be in the same folder as "robot_plotter.py". 

In [ ]:
from robot_plotter import init_plot, snapshot, show_plot
from math import sin, cos, atan2, sqrt
from random import random

# constants about the robot
robot_name = "Daneel"
robot_radius = 160
wheel_separation = 150
robot_wheel_radius = 35

# initial robot configuration
robot_x_position = 500
robot_y_position = 500
robot_heading = 0

#Map information
map_x_min = 0
map_x_max = 5000
map_y_min  = 0
map_y_max = 5000
map_coords = ((map_x_min, map_x_max), (map_y_min, map_y_max))
#This represents an obstacle at x = 100, y = 200, with a width of 500 and a height of 800.
obstacle_x = 100
obstacle_y = 2250
obstacle_width = 4000
obstacle_height = 500
obstacle = ((obstacle_x, obstacle_y), (obstacle_width, obstacle_height))
goal = (1000, 4000)

#Tell the plot function about the initial configuration of the robot.
init_plot(robot_x_position,robot_y_position,robot_heading)

#This is the number of timesteps we simulate- change as is necessary. Set to 5 in notebook so we don't get too many images. 
num_steps = 5
#This is the default timestep - so by default we'll simulate 5 timesteps of 1 second= 5 seconds of movement. 
delta_t= 1 

for ii in range(num_steps):

    #Random() will give us a random number between 0 and 1, so these lines will set the wheel speeds to be random numbers between 0 and 5.
    ang_speed_left = 5 * random()
    ang_speed_right = 5 * random()

    # convert angular speeds into linear speeds with linear_velocity = angular_velocity * radius
    linear_speed_left = ang_speed_left * robot_wheel_radius
    linear_speed_right = ang_speed_right * robot_wheel_radius
    # angular speed of the robot is given by the difference in speeds divided by wheel spacing
    ang_speed_robot = (linear_speed_left - linear_speed_right) / wheel_separation
    # multiply angular speed by time to get the amount the angle has changed.
    angle_change = ang_speed_robot * delta_t
    ave_speed = 0.5*(linear_speed_left + linear_speed_right)

    # sinc
    if angle_change**2<0.01**2:
        the_sinc = 1-(0.25*angle_change*angle_change/6.0)
    else:
        the_sinc = sin(0.5*angle_change)/(0.5*angle_change)

    # update state
    robot_x_position = robot_x_position + ave_speed*delta_t*the_sinc*sin(robot_heading + 0.5*angle_change)
    robot_y_position = robot_y_position + ave_speed*delta_t*the_sinc*cos(robot_heading + 0.5*angle_change)
    robot_heading = robot_heading + angle_change

    print("X position: ", robot_x_position, ", Y position:", robot_y_position, ", Heading: ", robot_heading)
    #Tell the plot function about the current configuration of the robot.
    snapshot(robot_x_position,robot_y_position,robot_heading)

    #Plot our current position for 0.1 seconds
    show_plot(map_coords, goal=goal, obstacle=obstacle, pause=0.1)


## Improving the simulator

Running robot_sim.py, you should see that the robot moves around randomly and is quite happy to move directly through the walls or obstacles. Not particularly useful!

<img src="https://raw.githubusercontent.com/engmaths/SEMT10002_2025/refs/heads/main/media/week_5/bad_robot.png" width="40%"/>

Our job today is to fix this- by adding code for goal-seeking, collision avoidance and eventually basic navigation.

### Goal-seeking

The first behaviour we'll add to our robot is the ability to move towards the goal. Algorithmically, we can break this down into smaller steps:

1. Turn until the robot faces the goal.
2. Move forward until the robot reaches the goal. 

To implement this algorithm in Python code, we'll need to break these bigger steps down further- that's left as a job for you. 

**Task** 

Add some code to robot_sim.py to let your robot move towards the goal. You'll need to do some trigonometry to do this, so I'd recommend you start by thinking about this problem with pen and paper first. You'll also probably want to use the atan2 function which computes the inverse tangent of two numbers. You can find official documentation for this function <a href=https://docs.python.org/3/library/math.html#math.atan2>here</a>, although I think the image at <a href=https://how.dev/answers/what-is-mathatan2-in-python>this link</a> will help you to visualise what's going on. Note that in our simulator, we've defined the vertical direction as equivalent to a heading of 0, so you'll need to provide the horizontal coordinate to atan2 first- i.e you should use atan2(x, y) rather than atan2(y, x) as seen in the examples. 

Once you've finished this task, and before you move on to the next steps, review your code to see if there are any places you could use functions to reduce repetition.

### Collision Avoidance

Now that we've got the robot moving towards the goal, let's look at collision avoidance. To do this, I suggest you turn off your goal-seeking code (commenting it out is an easy way to do this) and revert back to random movement. 

The core algorithmic idea behind collision avoidance is simple- before you move, check if the move will cause you to collide with something. If it will, then don't move. Otherwise, go ahead. 

We'll consider two kinds of objects that your robot might collide with:
 + walls (i.e bounds for the region the robot is moving around). These are specificed in code as map_coords = ((map_x_min, map_y_min), (map_x_max, map_y_max))
 + obstacles - regions within the map your robot cannot enter. Obstacles will always be rectangles and are defined in code as obstacle= ((obstacle_x, obstacle_y), (obstacle_width, obstacle_height)).

 **Task** 

Edit the provided code to include obstacle avoidance behaviour. It's upto you to decide exactly what your robot does when it hits an obstacle- the simplest thing would be to randomly choose a new direction of travel (by generating new wheel speeds), but this might cause your robot to get stuck against the obstacle- cleverer things are possible. I'd suggest you start by adding code for avoiding walls, then extend this to include code for avoiding obstacles. I'd also recommend that you ignore the robot's size for now and just consider it to be a point at (robot_x_position, robot_y_position). 

Once you've finished this task, and before you move on to the next steps, review your code to see if there are any places you could use functions to reduce repetition.

### Bug Algorithms

Now that we've got the two fundamental behaviours implemented, we can think about how to combine them into a single algorithm that can navigate to a goal in the presence of obstacles. To do this, we'll implement the *bug* algorithms. The *bug* family of algorithms are insect inspired navigation algorithms. We assume the robot can measure the distance and direction towards the goal, but otherwise only has sensors for detecting collisions. 

#### Bug 0

<img src="https://raw.githubusercontent.com/engmaths/SEMT10002_2025/refs/heads/main/media/week_5/bug0.png" width="41%"/>

The simplest bug algorithm is Bug 0, which is as follows:

1. Head towards goal until you reach the goal or hit an obstacle.
2. If you hit an obstacle, move around it until you can head towards the goal again.
3. Go to step 1.

To implement Bug 0, we'll first need to add another fundamental behaviour - wall following. There's a few different ways you could do this- in my solution, I broke it down into a horizontal wall following behaviour and a vertical wall following behaviour. For the horizontal wall following, I turned the robot to face horizontally (i.e heading = $\pi / 2$) and then drove forward. For vertical wall following, I did the same with a heading of 0 instead. 

Once you've got the wall following behaviours working, you'll need to figure out some conditions for when you should switch between them. Putting it all together, you should get something that looks like the image below:

<img src="https://raw.githubusercontent.com/engmaths/SEMT10002_2025/refs/heads/main/media/week_5/bug0_solution.png" width="41%"/>

**Task** 

Implement the bug 0 algorithm and test it on the provided map.

##### Better Bugs

Bug 0, while simple, can not solve all maps. There are two modifications of Bug 0 (known respectively as Bug 1 and Bug 2). 

The Bug 1 algorithm is as follows:

1. Head towards goal.
2. If an obstacle is encountered, circumnavigate it *and* remember which point on the perimeter is closest to the goal.
3. Return to the closest point (using wall-following) and then continue towards the goal.

<img src="https://raw.githubusercontent.com/engmaths/SEMT10002_2025/refs/heads/main/media/week_5/bug1.png" width="40%"/>

Bug 1 will solve more scenarios than Bug 1, but often produces very long paths (due to first circumnavigating each obstacle). Bug 2 attempts to solve this by introducing the concept of the *m-line*, the shortest line from start position to goal position (ignoring obstacles). Bug 2 is then:

1. Head towards goal on the m-line (our goal-seeking behaviour from above will do this for you!).
2. If an obstacle is in the way, follow it until you encounter the m-line closer to the goal than when you left it.
3. Leave obstacle and continue towards the goal.

<img src="https://raw.githubusercontent.com/engmaths/SEMT10002_2025/refs/heads/main/media/week_5/bug2.png" width="40%"/>

**Task** 

Implement one (or both if you're keen!) of the "better" bug algorithms.

